In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['bids.csv', 'train.csv', 'test.csv']


In [73]:
bid_data = pd.read_csv('../input/bids.csv')
print (bid_data.head(5))
train_data = pd.read_csv('../input/train.csv')
print (train_data.head(5))
test_data = pd.read_csv('../input/test.csv')
print (test_data.head(5))

   bid_id                              bidder_id auction merchandise  device  \
0       0  8dac2b259fd1c6d1120e519fb1ac14fbqvax8   ewmzr     jewelry  phone0   
1       1  668d393e858e8126275433046bbd35c6tywop   aeqok   furniture  phone1   
2       2  aa5f360084278b35d746fa6af3a7a1a5ra3xe   wa00e  home goods  phone2   
3       3  3939ac3ef7d472a59a9c5f893dd3e39fh9ofi   jefix     jewelry  phone4   
4       4  8393c48eaf4b8fa96886edc7cf27b372dsibi   jefix     jewelry  phone5   

               time country              ip              url  
0  9759243157894736      us   69.166.231.58  vasstdc27m7nks3  
1  9759243157894736      in   50.201.125.84  jmqlhflrzwuay9c  
2  9759243157894736      py  112.54.208.157  vasstdc27m7nks3  
3  9759243157894736      in   18.99.175.133  vasstdc27m7nks3  
4  9759243157894736      in    145.138.5.37  vasstdc27m7nks3  
                               bidder_id  \
0  91a3c57b13234af24875c56fb7e2b2f4rb56a   
1  624f258b49e77713fc34034560f93fb3hu3jo   
2  1c5f4f

In [74]:
def numberofActions(line, dataGrouped, dataid):
    if not line['bidder_id'] in dataid:
        return 0
    else:
        return dataGrouped[line['bidder_id']]

def mergingFeature(line, features, dataid):
    if not line['bidder_id'] in dataid:
        return 0
    else:
        index = features.index[features['bidder_id'] == line['bidder_id']]
        return features['value'][index].item()
    
def findMerchandise(line,grouped,dict,dataid):
    res = np.zeros(10)
    if line in dataid:
        merch = np.array(grouped[grouped['bidder_id']==line]['merchandise'])[0]
#         print merch
        res[dict[merch]] = 1
    return tuple(res)


In [75]:
def basicCountsPerUser():
    bidderList = bid_data['bidder_id'].unique()
    countryCount = bid_data['country'].groupby(bid_data['bidder_id']).count()
    ipCount = bid_data['ip'].groupby(bid_data['bidder_id']).count()
    urlCount = bid_data['url'].groupby(bid_data['bidder_id']).count()
    deviceCount = bid_data['device'].groupby(bid_data['bidder_id']).count()
    auctionCount = bid_data['auction'].groupby(bid_data['bidder_id']).count()
    grBidCount = bid_data['bid_id'].groupby(bid_data['bidder_id']).count()
    grMerchandiseCount = bid_data['merchandise'].groupby(bid_data['bidder_id']).count()
    payAccCountTrainList = train_data['payment_account'].groupby(train_data['bidder_id']).count()
    payAccCountTestList = test_data['payment_account'].groupby(test_data['bidder_id']).count()
    addressCountTrainList = train_data['address'].groupby(train_data['bidder_id']).count()
    addressCountTestList = test_data['address'].groupby(test_data['bidder_id']).count()

    train_data['nb0fCountry'] = train_data.apply(lambda x: numberofActions(x, countryCount, bidderList), axis=1)
    test_data['nb0fCountry'] = test_data.apply(lambda x: numberofActions(x, countryCount, bidderList), axis=1)

    train_data['nb0fIP'] = train_data.apply(lambda x: numberofActions(x, ipCount, bidderList), axis=1)
    test_data['nb0fIP'] = test_data.apply(lambda x: numberofActions(x, ipCount, bidderList), axis=1)

    train_data['nb0fURL'] = train_data.apply(lambda x: numberofActions(x, urlCount, bidderList), axis=1)
    test_data['nb0fURL'] = test_data.apply(lambda x: numberofActions(x, urlCount, bidderList), axis=1)

    train_data['nb0fDevice'] = train_data.apply(lambda x: numberofActions(x, deviceCount, bidderList), axis=1)
    test_data['nb0fDevice'] = test_data.apply(lambda x: numberofActions(x, deviceCount, bidderList), axis=1)

    train_data['nb0fAuction'] = train_data.apply(lambda x: numberofActions(x, auctionCount, bidderList), axis=1)
    test_data['nb0fAuction'] = test_data.apply(lambda x: numberofActions(x, auctionCount, bidderList), axis=1)

    train_data['nb0fBids'] = train_data.apply(lambda x: numberofActions(x, grBidCount, bidderList), axis=1)
    test_data['nb0fBids'] = test_data.apply(lambda x: numberofActions(x, grBidCount, bidderList), axis=1)

    train_data['nb0fMerch'] = train_data.apply(lambda x: numberofActions(x, grMerchandiseCount, bidderList), axis=1)
    test_data['nb0fMerch'] = test_data.apply(lambda x: numberofActions(x, grMerchandiseCount, bidderList), axis=1)

    train_data['nb0fPayAcc'] = train_data.apply(lambda x: numberofActions(x, payAccCountTrainList, bidderList), axis=1)
    test_data['nb0fPayAcc'] = test_data.apply(lambda x: numberofActions(x, payAccCountTestList, bidderList), axis=1)

    train_data['nb0fAdress'] = train_data.apply(lambda x: numberofActions(x, addressCountTrainList, bidderList), axis=1)
    test_data['nb0fAdress'] = test_data.apply(lambda x: numberofActions(x, addressCountTestList, bidderList), axis=1)
    

In [77]:
def bidsOnSelf():
    bidderList = bid_data['bidder_id'].unique()
    tmp_data = bid_data.sort_values(['auction', 'time']).groupby(bid_data['auction'])
    countBidsOnSelf = {}
    for t in tmp_data:
        prev = ''
        count = 0
        for b in t[1]['bidder_id']:
            if(b == prev):
                if(b in countBidsOnSelf.keys()):
                    count = countBidsOnSelf[b]
                    count += 1
            countBidsOnSelf[b] = count
            count = 0
            prev = b
    countBidsOnSelf = pd.DataFrame.from_dict(countBidsOnSelf)
    print (countBidsOnSelf)
    train_data['nb0fBidsOnSelf'] = train_data.apply(lambda x: mergingFeature(x, countBidsOnSelf, bidderList), axis=1)
    test_data['nb0fBidsOnSelf'] = test_data.apply(lambda x: mergingFeature(x, countBidsOnSelf, bidderList), axis=1)
#     print (train_data.head(5))

In [78]:

def basicUniqueCountsPerUser():
    bidderList = bid_data['bidder_id'].unique()
    countryCount = bid_data['country'].groupby(bid_data['bidder_id']).nunique()
    ipCount = bid_data['ip'].groupby(bid_data['bidder_id']).nunique()
    urlCount = bid_data['url'].groupby(bid_data['bidder_id']).nunique()
    deviceCount = bid_data['device'].groupby(bid_data['bidder_id']).nunique()
    auctionCount = bid_data['auction'].groupby(bid_data['bidder_id']).nunique()
    grBidCount = bid_data['bid_id'].groupby(bid_data['bidder_id']).nunique()
    grMerchandiseCount = bid_data['merchandise'].groupby(bid_data['bidder_id']).nunique()
    payAccCountTrainList = train_data['payment_account'].groupby(train_data['bidder_id']).nunique()
    payAccCountTestList = test_data['payment_account'].groupby(test_data['bidder_id']).nunique()
    addressCountTrainList = train_data['address'].groupby(train_data['bidder_id']).nunique()
    addressCountTestList = test_data['address'].groupby(test_data['bidder_id']).nunique()

    train_data['nb0fUniqueCountry'] = train_data.apply(lambda x: numberofActions(x, countryCount, bidderList), axis=1)
    test_data['nb0fUniqueCountry'] = test_data.apply(lambda x: numberofActions(x, countryCount, bidderList), axis=1)

    train_data['nb0fUniqueIP'] = train_data.apply(lambda x: numberofActions(x, ipCount, bidderList), axis=1)
    test_data['nb0fUniqueIP'] = test_data.apply(lambda x: numberofActions(x, ipCount, bidderList), axis=1)

    train_data['nb0fUniqueURL'] = train_data.apply(lambda x: numberofActions(x, urlCount, bidderList), axis=1)
    test_data['nb0fUniqueURL'] = test_data.apply(lambda x: numberofActions(x, urlCount, bidderList), axis=1)

    train_data['nb0fUniqueDevice'] = train_data.apply(lambda x: numberofActions(x, deviceCount, bidderList), axis=1)
    test_data['nb0fUniqueDevice'] = test_data.apply(lambda x: numberofActions(x, deviceCount, bidderList), axis=1)

    train_data['nb0fUniqueAuction'] = train_data.apply(lambda x: numberofActions(x, auctionCount, bidderList), axis=1)
    test_data['nb0fUniqueAuction'] = test_data.apply(lambda x: numberofActions(x, auctionCount, bidderList), axis=1)

    train_data['nb0fUniqueBids'] = train_data.apply(lambda x: numberofActions(x, grBidCount, bidderList), axis=1)
    test_data['nb0fUniqueBids'] = test_data.apply(lambda x: numberofActions(x, grBidCount, bidderList), axis=1)

    train_data['nb0fUniqueMerch'] = train_data.apply(lambda x: numberofActions(x, grMerchandiseCount, bidderList), axis=1)
    test_data['nb0fUniqueMerch'] = test_data.apply(lambda x: numberofActions(x, grMerchandiseCount, bidderList), axis=1)

    train_data['nb0fUniquePayAcc'] = train_data.apply(lambda x: numberofActions(x, payAccCountTrainList, bidderList), axis=1)
    test_data['nb0fUniquePayAcc'] = test_data.apply(lambda x: numberofActions(x, payAccCountTestList, bidderList), axis=1)

    train_data['nb0fUniqueAdress'] = train_data.apply(lambda x: numberofActions(x, addressCountTrainList, bidderList), axis=1)
    test_data['nb0fUniqueAdress'] = test_data.apply(lambda x: numberofActions(x, addressCountTestList, bidderList), axis=1)

In [79]:
def granularMerchandise():
    merchList = bid_data['merchandise'].unique()
#     print merchList
    train_data[['jewelry', 'furniture', 'home goods', 'mobile', 'sporting goods',
           'office equipment', 'computers', 'books and music', 'clothing',
           'auto parts']] = pd.DataFrame(np.zeros((train_data.shape[0], 10)),
                                         columns=['jewelry', 'furniture', 'home goods', 'mobile', 'sporting goods',
                                                  'office equipment', 'computers', 'books and music', 'clothing',
                                                  'auto parts'])

    test_data[['jewelry', 'furniture', 'home goods', 'mobile', 'sporting goods',
          'office equipment', 'computers', 'books and music', 'clothing',
          'auto parts']] = pd.DataFrame(np.zeros((train_data.shape[0], 10)),
                                        columns=['jewelry', 'furniture', 'home goods', 'mobile', 'sporting goods',
                                                 'office equipment', 'computers', 'books and music', 'clothing',
                                                 'auto parts'])

    grouped = bid_data[['bidder_id', 'merchandise']].drop_duplicates()
    res = train_data['bidder_id'].map(lambda x: findMerchandise(x, grouped, dict, merchList))
    (train_data['jewelry'], train_data['furniture'], train_data['home goods'],
     train_data['mobile'], train_data['sporting goods'], train_data['office equipment'],
     train_data['computers'], train_data['books and music'], train_data['clothing'], train_data['auto parts']) = zip(*res)
    res = test_data['bidder_id'].map(lambda x: findMerchandise(x, grouped, dict, merchList))
    (test_data['jewelry'], test_data['furniture'], test_data['home goods'],
     test_data['mobile'], test_data['sporting goods'], test_data['office equipment'],
     test_data['computers'], test_data['books and music'], test_data['clothing'], test_data['auto parts']) = zip(*res)

In [80]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

def testModel():
    print (train_data.head(5))
    basicCountsPerUser()
    basicUniqueCountsPerUser()
    granularMerchandise()
#     bidsOnSelf()
    tempTrainData = train_data
    X_train = tempTrainData.drop(["bidder_id", "outcome", "payment_account", "address"], axis=1)
    Y_train = tempTrainData["outcome"]
    logisticRegr = LogisticRegression()
    print (cross_val_score(logisticRegr, X_train, Y_train, cv=3).mean())


testModel()


                               bidder_id  \
0  91a3c57b13234af24875c56fb7e2b2f4rb56a   
1  624f258b49e77713fc34034560f93fb3hu3jo   
2  1c5f4fc669099bfbfac515cd26997bd12ruaj   
3  4bee9aba2abda51bf43d639013d6efe12iycd   
4  4ab12bc61c82ddd9c2d65e60555808acqgos1   

                         payment_account  \
0  a3d2de7675556553a5f08e4c88d2c228754av   
1  a3d2de7675556553a5f08e4c88d2c228v1sga   
2  a3d2de7675556553a5f08e4c88d2c2280cybl   
3  51d80e233f7b6a7dfdee484a3c120f3b2ita8   
4  a3d2de7675556553a5f08e4c88d2c22857ddh   

                                 address  outcome  
0  a3d2de7675556553a5f08e4c88d2c228vt0u4      0.0  
1  ae87054e5a97a8f840a3991d12611fdcrfbq3      0.0  
2  92520288b50f03907041887884ba49c0cl0pd      0.0  
3  4cb9717c8ad7e88a9a284989dd79b98dbevyi      0.0  
4  2a96c3ce94b3be921e0296097b88b56a7x1ji      0.0  
0.9448613000041309
